# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


# **Collect the top 10000 tweets by using hashtag "#blacklivesmatter" from Twitter.**

In [ ]:
# Write your code here

import tweepy as tw
my_api_key = "wYdO6UgfhcSAWp0V6z5xmxoaG"
my_api_secret = "3XjUWM8yIgVsHrxYfANwQ4cxuiwcq6KlWUjs9anFpymGldM2UD"

auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
import pandas as pd

search_query = "%23blacklivesmatter"

def getTweets():
  tweets = tw.Cursor(api.search,
              q=search_query,
              lang="en",
              since="2020-01-01").items(100)
  reviews = []
  for tweet in tweets:
    reviews.append(tweet)
  return reviews

def createTweetDataFrame():
  tweetReviews = getTweets()
  df = pd.DataFrame()
  for review in tweetReviews:
    hashtags = []
    try:
        for hashtag in review.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=review.id, tweet_mode='extended').full_text
    except:
        pass

    df = df.append(pd.DataFrame({'user_name': review.user.name, 
                                               'user_location': review.user.location,\
                                               'user_description': review.user.description,
                                               'user_verified': review.user.verified,
                                               'date': review.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': review.source}))
    df = df.reset_index(drop=True)
  return df

In [ ]:
df = createTweetDataFrame()
df

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,zoupdate,,Contemporary Artist Writer Photographer \nINAC...,False,2022-02-27 23:42:47,‘We’re like brothers and sisters’: Ukrainians ...,None,Twitter Web App
1,CruellaDeVos,New England,#Resister❤️seniors. Psych and Women’s Studies...,False,2022-02-27 23:42:45,RT @gutsywoman19: @stephhegarty @jelani9 No ma...,[BlackLivesMatter],Twitter for iPhone
2,Melonee Jinaki,Shalimar,PR.Comms.Journalist.Educator. Gulf Coast Missi...,False,2022-02-27 23:42:02,RT @scoopybenz: Ukrainian guards stop blacks f...,None,Twitter for Android
3,Sonam Malhotra,NASA,Here for fun|Let's Be Friends 🤝|Live and Let L...,False,2022-02-27 23:42:01,RT @sonammalhotra27: Live videos available fro...,None,Twitter for Android
4,Sonam Malhotra,NASA,Here for fun|Let's Be Friends 🤝|Live and Let L...,False,2022-02-27 23:41:48,Live videos available from Ukraine too. Stop y...,None,Twitter for Android
...,...,...,...,...,...,...,...,...
95,Harish Srinivasa,,"Talks straight, when you speak truth, you need...",False,2022-02-27 23:25:03,RT @DrBirendraShar2: What's going around in Uk...,None,Twitter for Android
96,hypatia264,in superposition,Retired CIA officer and currently a digital ga...,False,2022-02-27 23:24:38,You know how citizens' videos of police have c...,None,Twitter Web App
97,Askin,Chengdu,"son, brother, uncle and an advocate for equal ...",False,2022-02-27 23:24:33,@courtines @AlanRMacLeod @kthalps You can't ev...,None,Twitter for Android
98,lina seitz,,,False,2022-02-27 23:24:28,"Racism Claims Emerge Out of Ukraine, Africans ...","[racism, racist, blacklivesmatter, blm]",Twitter for iPhone


In [ ]:
df.to_csv('/content/sample_data/Twitter.csv')

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [ ]:
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
import nltk
import numpy as np
import pandas as pd
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

nltk.download('stopwords')
nltk.download('punkt')

sp = spacy.load('en_core_web_sm')
tokenizer = ToktokTokenizer()
stopwords_list = stopwords.words('english')
nltk.download('averaged_perceptron_tagger')


In [ ]:
from pandas.io.xml import preprocess_data

def basicPreprocessing(filepath):
  df = pd.read_csv(filepath)
  df.replace(np.nan, '', inplace=True)
  textList = df['text'].values.tolist()
  preprocessedTextList = []
  for text in textList:
    combinedString = expandContractions(text)
    combinedString = lowerCase(combinedString)
    combinedString = removeSpecialCharcters(combinedString)
    combinedString = removeNumbers(combinedString)
    combinedString = remove_stopwords(combinedString)
    combinedString = lemmatizeText(combinedString)
    combinedString = stemming(combinedString)
    preprocessedTextList.append(combinedString)
  df['Preporcessed_Text'] = preprocessedTextList
  df.to_csv('/content/sample_data/PreprocessedReviews.csv')

def expandContractions(combinedText):
  words = combinedText.split()
  expanded_words_list = [CONTRACTION_MAP[word] if word in CONTRACTION_MAP else word for word in words]
  expanded_text = ' '.join(expanded_words_list)
  return expanded_text

def removeSpecialCharcters(combinedText):
  try:
    pattern = r'[^a-zA-Z0-9\s]'
    combinedText = re.sub(pattern, '', combinedText)
  except TypeError:
    pass
  return combinedText

def removeNumbers(combinedText):
 return re.sub(r'[~^0-9]', '', combinedText)

def get_tokens(combinedText):
  tokens = tokenizer.tokenize(combinedText)
  tokens = [token.strip() for token in tokens]
  return tokens

def remove_stopwords(combinedText):
  tokens = get_tokens(combinedText)
  filtered_tokens = [token for token in tokens if token.lower() not in stopwords_list]
  filtered_text = ' '.join(filtered_tokens)
  return filtered_text 

def lowerCase(combinedText):
  return combinedText.lower()

def lemmatizeText(combinedText):
  try:
    combinedText = sp(combinedText)
    combinedText = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in combinedText])
  except TypeError: pass
  return combinedText

def stemming(combinedText):
  st = PorterStemmer()
  words = word_tokenize(combinedText)
  wordList = []
  for w in words:
    wordList.append(w)
  return " ".join(wordList)
    

In [ ]:
basicPreprocessing('/content/sample_data/Twitter.csv')

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
!pip install benepar

In [ ]:
import spacy
nltk.download('punkt')
from spacy import displacy
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, pos_tag_sents
import benepar

parser=benepar.Parser("benepar_en3")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def POS(filepath):
  df = pd.read_csv(filepath)
  df.replace(np.nan, '', inplace=True)
  preprocessed_List = df['Preporcessed_Text'].values.tolist()
  posList = []
  for text in preprocessed_List:
    text = word_tokenize(text)
    posList.append(nltk.pos_tag(text))
  
  return posList

def countPOS(filepath):
  pos = POS(filepath)
  count = []
  for tags in pos:
    counts = Counter(tag for words, tag in tags)
    count.append(counts)
  return count

def parsingTree(filepath):
  df = pd.read_csv(filepath)
  df.replace(np.nan, '', inplace=True)
  preprocessed_List = df['Preporcessed_Text'].values.tolist()
  for statement in preprocessed_List: 
    try:
      tree = parser.parse(statement)
      print(tree)
    except:
      print("No Parse Tree")
      continue

def dependencyParsing(filepath):
  df = pd.read_csv(filepath)
  df.replace(np.nan, '', inplace=True)
  preprocessed_List = df['Preporcessed_Text'].values.tolist()
  nlp = spacy.load('en')
  options = {'compact': True, 'font' : 'Source Sans Pro' , 'distance' : 100}
  for each_sentence in preprocessed_List:
    doc = nlp(each_sentence)
    displacy.render(doc,style = 'dep', options= options, jupyter = True)

In [ ]:
%tensorflow_version 1.x
import benepar
benepar.download('benepar_en3')
parsing = benepar.Parser("benepar_en3")

[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()

def namedEntity(filepath):
  df = pd.read_csv(filepath)
  df.replace(np.nan, '', inplace=True)
  preprocessed_List = df['Preporcessed_Text'].values.tolist()
  for sentence in preprocessed_List:
    doc_file=nlp(sentence)
    for i in doc_file.ents:
      if i.text and i.label_:
        print([(i.text, i.label_)])

In [ ]:
countPOS('/content/sample_data/PreprocessedReviews.csv')

[Counter({'IN': 1,
          'JJ': 2,
          'JJR': 1,
          'NN': 9,
          'NNPS': 1,
          'NNS': 1,
          'VB': 1,
          'VBP': 1,
          'VBZ': 1}),
 Counter({'NN': 7, 'VBP': 1}),
 Counter({'JJ': 4, 'NN': 9, 'RB': 1, 'VB': 2, 'VBP': 1}),
 Counter({'FW': 1, 'JJ': 6, 'JJS': 1, 'NN': 8}),
 Counter({'FW': 1, 'JJ': 5, 'JJS': 1, 'NN': 11, 'VB': 1}),
 Counter({'IN': 2,
          'JJ': 4,
          'NN': 12,
          'PRP': 1,
          'RB': 5,
          'TO': 1,
          'VB': 2,
          'VBN': 1,
          'VBP': 4}),
 Counter({'JJ': 1, 'NN': 9, 'RB': 1, 'VB': 1}),
 Counter({'JJ': 2, 'NN': 8, 'VBD': 1}),
 Counter({'IN': 1, 'JJ': 3, 'JJR': 1, 'NN': 6, 'NNS': 1, 'VB': 1, 'VBP': 2}),
 Counter({'JJ': 1,
          'NN': 4,
          'NNS': 1,
          'PRP': 1,
          'RB': 2,
          'VB': 1,
          'VBD': 1,
          'WRB': 1}),
 Counter({'IN': 1, 'JJ': 1, 'NN': 9, 'VBP': 1}),
 Counter({'JJ': 3, 'NN': 6, 'VBP': 2}),
 Counter({'JJ': 2,
          'JJR'

In [ ]:
parsingTree('/content/sample_data/PreprocessedReviews.csv')

/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


(TOP
  (S
    (PP
      (PP (IN like) (NP (NN brother) (NN sister)))
      (NNS ukrainians))
    (NP (NNS russians))
    (VP
      (VBP unite)
      (NNP london)
      (VP (VB protest) (NP (NNS putins) (NN war)))
      (FW httpstcokuxtbzbo)
      (NN holocaustmemorialday))
    (NN mlk)
    (NN grenfelltower)
    (NN blacklivesmatter)
    (FW bidenharris)
    (NN oprahmeghanharry)
    (RB reparationsnow)))
(TOP
  (S
    (NP
      (RB rt)
      (NN gutsywoman)
      (NN stephhegarty)
      (FW jelani)
      (NN matter))
    (VP (VB go) (NP (NN world) (NN blacklivesmatter)))))
(TOP
  (S
    (RB rt)
    (NN scoopybenz)
    (JJ ukrainian)
    (NN guard)
    (ADJP (VB stop) (ADJP (JJ black)))
    (VB get)
    (NP (NN train))
    (VP
      (VP (VB flee) (NP (NN war) (NN ukraine)))
      (VP
        (VB tell)
        (S
          (VP
            (ADJP (VB be) (ADJP (RB not) (VBN surprised)))
            (S (VP (VB see) (ADVP (JJ th))))))))))
(TOP
  (NP
    (NP (FW rt) (FW sonammalhotra) (NP (J

In [ ]:
dependencyParsing('/content/sample_data/PreprocessedReviews.csv')


In [ ]:
namedEntity('/content/sample_data/PreprocessedReviews.csv')

[('russians', 'NORP')]
[('london', 'GPE')]
[('mlk', 'ORG')]
[('bidenharris oprahmeghanharry', 'PERSON')]
[('scoopybenz ukrainian guard', 'ORG')]
[('ukraine', 'GPE')]
[('ukraine', 'GPE')]
[('west white', 'GPE')]
[('ukraine', 'GPE')]
[('west white', 'GPE')]
[('africansinukraine racism', 'PERSON')]
[('ccpchina httpstcorgdohdyyrn', 'PERSON')]
[('russian', 'NORP')]
[('ukraine', 'GPE')]
[('london', 'GPE')]
[('uk', 'GPE')]
[('mlk', 'ORG')]
[('bidenharris oprahmeghanharry', 'PERSON')]
[('chickalettas avenattimary', 'ORG')]
[('splcenter kaepernick', 'PERSON')]
[('rt zodiacdka', 'PERSON')]
[('african', 'NORP')]
[('brainwash westerner', 'PERSON')]
[('russia', 'GPE')]
[('africa', 'LOC')]
[('russian', 'NORP')]
[('hundred', 'CARDINAL')]
[('ukraine', 'GPE')]
[('holocaustmemorialday', 'DATE')]
[('mlk', 'ORG')]
[('bidenharris oprahmeghanharry', 'PERSON')]
[('dartagnan musketeer', 'PERSON')]
[('xbox xboxambassadors', 'PERSON')]
[('hmu', 'ORG')]
[('lets go', 'PERSON')]
[('putin', 'PERSON')]
[('blacklives

**Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Syntatical parsing is another name for constituency parsing.
It is the division of a text into sub-texts or sub-paragraphs, each of which is referred to as a constituent.
It specifies a string's or sentence's syntactical structure.
These sub-categories of grammar are known as Noun-Phrase(NP) and Verb-Phrase(VP) (VP). 

Example: 

It took me over two hours to finish the translation.
(S (NP (NP (PRP It)) (VP (VBD took)) (NP (PRP me)) (NP (QR (GGR more) IN than) (CD two)) 
(NNS hours)) (S(VP (TO to)) (VP (VB translate)) (S(VP (TO to)) (S(VP (TO to)) (S(VP (TO to))
(S(VP (TO to)) (S(VP (TO to)).

Dependency Parsing Tree: Dependency parsing is the process of assessing a sentence's grammatical structure based on the dependencies of its words. Example: Weather that is rainy The adjective rainy affects the meaning of the noun weather in this statement. As a result, there is a link between wet days and the weather. As a result, the weather plays the role of a parent, while the rainy season plays the role of a child.